In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:
#path_df = "Review_dataset.pickle"

In [3]:
#with open(path_df, 'rb') as data:
df = pd.read_csv("df_more20.csv")

In [4]:
df.head()

,Reviewer_Nationality,Negative_Review,Review_length
0,Australia,Rooms are nice but for elderly a bit difficul...,206
1,Australia,We had issues with our electronic key everyda...,378
2,Australia,it s a shame about all the construction going...,122
3,Australia,Foyer was a mess Only place to relax was the ...,923
4,Australia,Room could have done with a microwave our roo...,117


In [5]:
#1. Text cleaning and preparation
# \r and \n
df['Content_Parsed_1'] = df['Negative_Review'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

In [6]:
# " when quoting text
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [7]:
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [8]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

In [9]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

In [10]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jcaravella\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jcaravella\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [12]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [13]:
df['Content_Parsed_5'] = lemmatized_text_list

In [14]:
##Stop Words# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcaravella\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [16]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [17]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

In [18]:
df.head(3)

,Reviewer_Nationality,Negative_Review,Review_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,Australia,Rooms are nice but for elderly a bit difficul...,206,Rooms are nice but for elderly a bit difficul...,rooms are nice but for elderly a bit difficul...,rooms are nice but for elderly a bit difficul...,rooms are nice but for elderly a bit difficul...,room be nice but for elderly a bite difficult...,room nice elderly bite difficult room ...
1,Australia,We had issues with our electronic key everyda...,378,We had issues with our electronic key everyda...,we had issues with our electronic key everyda...,we had issues with our electronic key everyda...,we had issues with our electronic key everyda...,we have issue with our electronic key everyda...,issue electronic key everyday deactivate...
2,Australia,it s a shame about all the construction going...,122,it s a shame about all the construction going...,it s a shame about all the construction going...,it s a shame about all the construction going...,it s a shame about all the construction going...,it s a shame about all the construction go on...,shame construction go look like be...


In [19]:
list_columns = ["Reviewer_Nationality", "Negative_Review", "Content_Parsed_6"]
df = df[list_columns]
df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
df.head()

,Reviewer_Nationality,Negative_Review,Content_Parsed
0,Australia,Rooms are nice but for elderly a bit difficul...,room nice elderly bite difficult room ...
1,Australia,We had issues with our electronic key everyda...,issue electronic key everyday deactivate...
2,Australia,it s a shame about all the construction going...,shame construction go look like be...
3,Australia,Foyer was a mess Only place to relax was the ...,foyer mess place relax bar comforta...
4,Australia,Room could have done with a microwave our roo...,room could microwave room small leak ...


In [20]:

#2. Label coding

In [21]:
category_codes = {
    'Australia': 0,
    'Canada': 1, 
 'Germany': 2, 
 'Ireland': 3,
 'Netherlands': 4, 
 'Saudi Arabia': 5,
 'Switzerland': 6, 
 'United Arab Emirates': 7,
 'United Kingdom': 8,
 'United States of America': 9
}

In [22]:
# Category mapping
df['Category_Code'] = df['Reviewer_Nationality']
df = df.replace({'Category_Code':category_codes})

In [23]:
df.head()

,Reviewer_Nationality,Negative_Review,Content_Parsed,Category_Code
0,Australia,Rooms are nice but for elderly a bit difficul...,room nice elderly bite difficult room ...,0
1,Australia,We had issues with our electronic key everyda...,issue electronic key everyday deactivate...,0
2,Australia,it s a shame about all the construction going...,shame construction go look like be...,0
3,Australia,Foyer was a mess Only place to relax was the ...,foyer mess place relax bar comforta...,0
4,Australia,Room could have done with a microwave our roo...,room could microwave room small leak ...,0


In [24]:
##3. Train - test split

In [26]:
target = df["Category_Code"]
target_names = ['United Kingdom','United States of America', 'Australia', 'Ireland', 'United Arab Emirates','Saudi Arabia', 'Netherlands','Switzerland','Germany','Canada']

In [27]:
data = df.drop("Category_Code", axis=1)
feature_names = data.columns
data.head()

,Reviewer_Nationality,Negative_Review,Content_Parsed
0,Australia,Rooms are nice but for elderly a bit difficul...,room nice elderly bite difficult room ...
1,Australia,We had issues with our electronic key everyda...,issue electronic key everyday deactivate...
2,Australia,it s a shame about all the construction going...,shame construction go look like be...
3,Australia,Foyer was a mess Only place to relax was the ...,foyer mess place relax bar comforta...
4,Australia,Room could have done with a microwave our roo...,room could microwave room small leak ...


In [37]:
data.to_csv(r'C:\Users\jcaravella\Documents\Finalproject\parsed.csv', index = False,header=True)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)


In [33]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [34]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(154445, 300)
(27255, 300)


In [35]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'Australia' category:
  . Most correlated unigrams:
. wifi
. bag
. park
. fridge
. luggage
  . Most correlated bigrams:
. room little
. air con

# 'Canada' category:
  . Most correlated unigrams:
. street
. bar
. luggage
. front
. desk
  . Most correlated bigrams:
. air con
. tea coffee

# 'Germany' category:
  . Most correlated unigrams:
. windows
. city
. park
. star
. loud
  . Most correlated bigrams:
. tea coffee
. star hotel

# 'Ireland' category:
  . Most correlated unigrams:
. food
. expensive
. city
. park
. bar
  . Most correlated bigrams:
. room service
. double bed

# 'Netherlands' category:
  . Most correlated unigrams:
. something
. tire
. wifi
. star
. old
  . Most correlated bigrams:
. small room
. star hotel

# 'Saudi Arabia' category:
  . Most correlated unigrams:
. facilities
. extremely
. extra
. experience
. expect
  . Most correlated bigrams:
. double bed
. would nice

# 'Switzerland' category:
  . Most correlated unigrams:
. city
. street
. windows
. old
. star


In [36]:

##Export what is needed for the next step
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)